In [1]:
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sb

# Read Data

In [2]:
train=pd.read_csv("C:/Users/AYUSH/Desktop/Dataset/Diabetes Dataset/train.csv")

# Profile

In [3]:
train.head()

,p_id,no_times_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,bmi,diabetes pedigree,age,diabetes
0,316,2,112,68,22,94,34.1,0.315,26,0
1,25,11,143,94,33,146,36.6,0.254,51,1
2,710,2,93,64,32,160,38.0,0.674,23,1
3,658,1,120,80,48,200,38.9,1.162,41,0
4,542,3,128,72,25,190,32.4,0.549,27,1


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   p_id                   614 non-null    int64  
 1   no_times_pregnant      614 non-null    int64  
 2   glucose_concentration  614 non-null    int64  
 3   blood_pressure         614 non-null    int64  
 4   skin_fold_thickness    614 non-null    int64  
 5   serum_insulin          614 non-null    int64  
 6   bmi                    614 non-null    float64
 7   diabetes pedigree      614 non-null    float64
 8   age                    614 non-null    int64  
 9   diabetes               614 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 48.1 KB


In [5]:
train.describe()

,p_id,no_times_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,bmi,diabetes pedigree,age,diabetes
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,385.773616,3.853420,120.542345,68.765472,20.244300,79.355049,31.909935,0.466342,33.325733,0.348534
std,223.603024,3.358126,31.252286,19.914836,15.886083,117.709950,8.007699,0.331090,11.929569,0.476895
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,191.250000,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.240250,24.000000,0.000000
50%,387.000000,3.000000,117.000000,72.000000,23.000000,17.000000,32.000000,0.361000,29.000000,0.000000
75%,572.750000,6.000000,139.000000,80.000000,32.000000,126.000000,36.600000,0.613500,41.000000,1.000000
max,768.000000,17.000000,197.000000,114.000000,63.000000,846.000000,59.400000,2.420000,81.000000,1.000000


# Find missing data

In [6]:
train.isna().sum()

p_id                     0
no_times_pregnant        0
glucose_concentration    0
blood_pressure           0
skin_fold_thickness      0
serum_insulin            0
bmi                      0
diabetes pedigree        0
age                      0
diabetes                 0
dtype: int64

# Drop unnessary columns

In [7]:
train = train.drop(labels="p_id",axis=1)

# Define X and Y

In [8]:
X = train.drop(labels="diabetes",axis=1)
Y = train[["diabetes"]]

# Find and Remove Skew

In [9]:
X.skew().sort_values()

blood_pressure          -1.877868
bmi                     -0.539151
skin_fold_thickness      0.014201
glucose_concentration    0.120295
no_times_pregnant        0.906273
age                      1.143901
diabetes pedigree        1.978319
serum_insulin            2.394700
dtype: float64

In [10]:
cols_skew = ["no_times_pregnant","age","diabetes pedigree","serum_insulin"]

In [11]:
for i in cols_skew:
    for j in range(0,614,1):
        if(X[[i]].iloc[j,0]==0):
            X[[i]] = X[[i]].replace(0,0.001)
X1 = X[cols_skew].apply(np.log)

In [12]:
X = X.drop(labels=cols_skew,axis=1)
X2 = X.join(X1)

In [13]:
X2.head(2)

,glucose_concentration,blood_pressure,skin_fold_thickness,bmi,no_times_pregnant,age,diabetes pedigree,serum_insulin
0,112,68,22,34.1,0.693147,3.258097,-1.155183,4.543295
1,143,94,33,36.6,2.397895,3.931826,-1.370421,4.983607


# Preprocessing

In [14]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_new = pd.DataFrame(ss.fit_transform(X2),columns=X2.columns)

In [15]:
X_new.head(3)

,glucose_concentration,blood_pressure,skin_fold_thickness,bmi,no_times_pregnant,age,diabetes pedigree,serum_insulin
0,-0.273558,-0.038469,0.110608,0.273718,0.221589,-0.590595,-0.284440,0.941567
1,0.719178,1.268155,0.803603,0.586172,0.795984,1.480039,-0.619645,1.016456
2,-0.882009,-0.239488,0.740603,0.761146,0.221589,-0.967400,0.900180,1.032029


# Spliting data into training and testing set

In [16]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X_new,Y,test_size=0.2,random_state=42)

# Create Neural Network

In [17]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

nn = Sequential()
nn.add(Dense(128,activation="relu",input_dim = X_new.shape[1]))
nn.add(Dropout(0.2))
nn.add(Dense(128,activation="relu"))
nn.add(Dropout(0.2))
nn.add(Dense(128,activation="relu"))
nn.add(Dropout(0.2))
nn.add(Dense(1,activation="sigmoid"))

In [18]:
nn.compile(loss="binary_crossentropy",optimizer="Adam",metrics=["accuracy"])
nn.fit(X_new,Y,validation_split=0.2,epochs=50)

Epoch 1/50
16/16 [==============================] - 1s 14ms/step - loss: 0.6340 - accuracy: 0.6334 - val_loss: 0.5877 - val_accuracy: 0.6992
Epoch 2/50
16/16 [==============================] - 0s 3ms/step - loss: 0.5336 - accuracy: 0.7291 - val_loss: 0.5410 - val_accuracy: 0.7561
Epoch 3/50
16/16 [==============================] - 0s 3ms/step - loss: 0.4832 - accuracy: 0.7678 - val_loss: 0.5649 - val_accuracy: 0.7642
Epoch 4/50
16/16 [==============================] - 0s 3ms/step - loss: 0.4502 - accuracy: 0.7862 - val_loss: 0.5683 - val_accuracy: 0.7561
Epoch 5/50
16/16 [==============================] - 0s 3ms/step - loss: 0.4598 - accuracy: 0.8024 - val_loss: 0.5724 - val_accuracy: 0.7642
Epoch 6/50
16/16 [==============================] - 0s 3ms/step - loss: 0.4254 - accuracy: 0.8147 - val_loss: 0.5753 - val_accuracy: 0.7480
Epoch 7/50
16/16 [==============================] - 0s 4ms/step - loss: 0.4420 - accuracy: 0.7862 - val_loss: 0.5772 - val_accuracy: 0.7561
Epoch 8/50
16/16 [=

# Tuning model parameters

In [19]:
def model(hp):
    nn = Sequential()
    nn.add(Dense(128,activation="relu",input_dim = X_new.shape[1]))
    x = hp.Int("units",min_value=128,max_value=640,step=32)
    y = hp.Choice("rate",values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
    nn.add(Dropout(rate=y))
    nn.add(Dense(units = x,activation="relu"))
    nn.add(Dropout(rate=y))
    nn.add(Dense(units = x,activation="relu"))
    nn.add(Dropout(rate=y))
    nn.add(Dense(1,activation="sigmoid"))
    nn.compile(loss="binary_crossentropy",metrics=["accuracy"])
    return nn

In [20]:
import keras_tuner as kt
tuner = kt.Hyperband(model,
                     objective="val_accuracy",
                    max_epochs=10,
                    directory="C:/Users/AYUSH/Desktop/Keras_Tuner/KT_NN3",
                    project_name="Keras_TUNER")


In [21]:
tuner.search(xtrain,ytrain,epochs=60,validation_split=0.2)

Trial 30 Complete [00h 00m 02s]
val_accuracy: 0.6767676472663879

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 45s
INFO:tensorflow:Oracle triggered exit


In [22]:
tuner.get_best_hyperparameters()[0].get("units")

160

In [23]:
tuner.get_best_hyperparameters()[0].get("rate")

0.3

# Create a NN with new value of tuned parameter

In [43]:
nn1 = Sequential()
nn1.add(Dense(128,activation="relu",input_dim = X_new.shape[1]))
nn1.add(Dropout(0.3))
nn1.add(Dense(160,activation="relu"))
nn1.add(Dropout(0.3))
nn1.add(Dense(160,activation="relu"))
nn1.add(Dropout(0.3))
nn1.add(Dense(1,activation="sigmoid"))

In [44]:
nn1.compile(loss="binary_crossentropy",optimizer="Adam",metrics=["accuracy"])
nn1.fit(X_new,Y,validation_split=0.2,epochs=50)

Epoch 1/50
16/16 [==============================] - 1s 12ms/step - loss: 0.6277 - accuracy: 0.6395 - val_loss: 0.5475 - val_accuracy: 0.7805
Epoch 2/50
16/16 [==============================] - 0s 4ms/step - loss: 0.5219 - accuracy: 0.7556 - val_loss: 0.5225 - val_accuracy: 0.7886
Epoch 3/50
16/16 [==============================] - 0s 4ms/step - loss: 0.4719 - accuracy: 0.7800 - val_loss: 0.5422 - val_accuracy: 0.7642
Epoch 4/50
16/16 [==============================] - 0s 4ms/step - loss: 0.4463 - accuracy: 0.7821 - val_loss: 0.5609 - val_accuracy: 0.7642
Epoch 5/50
16/16 [==============================] - 0s 4ms/step - loss: 0.4630 - accuracy: 0.7862 - val_loss: 0.5667 - val_accuracy: 0.7398
Epoch 6/50
16/16 [==============================] - 0s 4ms/step - loss: 0.4409 - accuracy: 0.8065 - val_loss: 0.5745 - val_accuracy: 0.7236
Epoch 7/50
16/16 [==============================] - 0s 4ms/step - loss: 0.4338 - accuracy: 0.8024 - val_loss: 0.5779 - val_accuracy: 0.7642
Epoch 8/50
16/16 [=

In [45]:
w = []
for i in nn1.predict(xtest):
    if(i[0]<0.5):
        w.append(0)
    else:
        w.append(1)

In [46]:
w[:6]

[0, 0, 0, 1, 0, 0]

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,w)

0.8211382113821138

# Prediction

In [48]:
test = pd.read_csv("C:/Users/AYUSH/Desktop/Dataset/Diabetes Dataset/test.csv")

In [49]:
test.head()

,p_id,no_times_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,bmi,diabetes pedigree,age
0,437,12,140,85,33,0,37.4,0.244,41
1,411,6,102,90,39,0,35.7,0.674,28
2,639,7,97,76,32,91,40.9,0.871,32
3,213,7,179,95,31,0,34.2,0.164,60
4,181,6,87,80,0,0,23.2,0.084,32


In [50]:
Test = test.drop(labels="p_id",axis=1)

In [51]:
test.isna().sum()

p_id                     0
no_times_pregnant        0
glucose_concentration    0
blood_pressure           0
skin_fold_thickness      0
serum_insulin            0
bmi                      0
diabetes pedigree        0
age                      0
dtype: int64

In [52]:
T1 = pd.DataFrame(ss.fit_transform(Test),columns=Test.columns)

In [53]:
T1.head()

,no_times_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,bmi,diabetes pedigree,age
0,2.398045,0.510766,0.861270,0.699161,-0.778084,0.689682,-0.754295,0.732529
1,0.640873,-0.585713,1.157464,1.070440,-0.778084,0.458788,0.543396,-0.443512
2,0.933735,-0.729986,0.328121,0.637281,0.089936,1.165051,1.137920,-0.081653
3,0.933735,1.636099,1.453658,0.575402,-0.778084,0.255059,-0.995726,2.451357
4,0.640873,-1.018533,0.565076,-1.342871,-0.778084,-1.238958,-1.237157,-0.081653


In [54]:
final_Pred = []
for i in nn1.predict(T1):
    if(i<0.5):
        final_Pred.append(0)
    else:
        final_Pred.append(1)
    

In [55]:
final_Pred[:5]

[1, 1, 1, 0, 0]

In [56]:
test["diabetes"] = final_Pred

In [57]:
test.head()

,p_id,no_times_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,bmi,diabetes pedigree,age,diabetes
0,437,12,140,85,33,0,37.4,0.244,41,1
1,411,6,102,90,39,0,35.7,0.674,28,1
2,639,7,97,76,32,91,40.9,0.871,32,1
3,213,7,179,95,31,0,34.2,0.164,60,0
4,181,6,87,80,0,0,23.2,0.084,32,0


In [58]:
test[["p_id","diabetes"]].to_csv("C:/Users/AYUSH/Desktop/Dataset/Diabetes Dataset/output.csv")